# MEOP dataset preprocessing 2018 release
MEOP data can be found at: http://www.meop.net/database/format.html
MEOP stands for: "Marine Mammals Exploring the Oceans from Pole to Pole".
MEOP consists of ARGO like profiles, however, the instruments are fitted onto marine mammals.
The UK dataset of MEOP contains many profiles in the Weddel Sea region, the following analysis is of this dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gsw
from netCDF4 import Dataset
import xarray as xr
import pdb
from IPython.display import Image
from scipy.interpolate import griddata
from scipy.io import netcdf
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial.distance import cdist
from haversine import haversine

/home/km/anaconda2/envs/py35new/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


Make sure that all the paths to the netcdf files are given in a column in the file named "filenames.txt"

This can be done by 
1. navigating to the myScripts folder in the terminal and typing in for eg. 
: "ls ../USA/DATA_ncARGO/ |grep .nc | xargs -i echo "../USA/DATA_ncARGO/{}" >> filenames.txt"

### The following is the creation of the merged csv file from the separate netcdf files. The Pandas package is able to efficiently read the csv file and then perform operations on the data.  Only the countries with southern hemisphere profiles are selected in the "filenames.txt" list.

An alternative and perhaps more efficient way of processing the data would be to write it all in Xarrays. However, since I developed all the original codes and scripts for plotting etc. in pandas, I am sticking with pandas for **legacy** support reasons.

The following operation is time consuming (should take 2 - 3 hrs at the most, but ensure at least 16GB RAM), but it is a one time operation. It requires a workstation with sufficient memory to read and write all the data. The CSV file has already been prepared and saved to disk, you can use that directly.

In [11]:
filenamesMerged = pd.read_csv("filenames.txt", header=None)

In [3]:
def create_merged_df(filenames):
    for i in range((len(filenames))):
        arr_data = xr.open_dataset(filenames[0][i])
        nlev = len(arr_data['PRES_ADJUSTED'][0])
        nprof = len(arr_data['PRES_ADJUSTED'])
        if(i == 0):
            NPROF = np.arange(nprof)
            NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])
        else:
            last_end = NPROF.flatten()[-1] + 1
            NPROF = np.arange(last_end, last_end+nprof )
            NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])
        
        ind = np.arange(nprof*nlev)
        lat = np.array([[arr_data["LATITUDE"][j].values]* nlev for j in range(nprof) ])
        lon = np.array([[arr_data["LONGITUDE"][j].values]* nlev for j in range(nprof) ])
        posqc = np.array([[arr_data["POSITION_QC"][j].values]* nlev for j in range(nprof) ])
        juld = np.array([[arr_data["JULD"][j].values]* nlev for j in range(nprof) ])
        
        if(i == 0):    
            df = {'PLATFORM_NUMBER': pd.Series([str(arr_data["PLATFORM_NUMBER"][0].values)]*len(ind), index=ind), 
                  'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                  'TEMP_ADJUSTED': pd.Series(arr_data["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED': pd.Series(arr_data["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED': pd.Series(arr_data["PRES_ADJUSTED"].values.flatten(), index=ind),  
                  'PRES_ADJUSTED_QC': pd.Series(arr_data["PRES_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED_ERROR':pd.Series(arr_data["PRES_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                  'TEMP_ADJUSTED_QC': pd.Series(arr_data["TEMP_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'TEMP_ADJUSTED_ERROR': pd.Series(arr_data["TEMP_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED_QC': pd.Series(arr_data["PSAL_ADJUSTED_QC"].values.flatten(), index=ind),  
                  'PSAL_ADJUSTED_ERROR':pd.Series(arr_data["PSAL_ADJUSTED_ERROR"].values.flatten(), index=ind),  
                  'JULD': pd.Series(juld.flatten(), index=ind),  
                  'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                  'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                  'POSITION_QC': pd.Series(posqc.flatten(), index=ind) }
            df = pd.DataFrame(df)
        else:
            df_i = {'PLATFORM_NUMBER': pd.Series([str(arr_data["PLATFORM_NUMBER"][0].values)]*len(ind), index=ind), 
                  'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                  'TEMP_ADJUSTED': pd.Series(arr_data["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED': pd.Series(arr_data["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED': pd.Series(arr_data["PRES_ADJUSTED"].values.flatten(), index=ind),  
                  'PRES_ADJUSTED_QC': pd.Series(arr_data["PRES_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED_ERROR':pd.Series(arr_data["PRES_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                  'TEMP_ADJUSTED_QC': pd.Series(arr_data["TEMP_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'TEMP_ADJUSTED_ERROR': pd.Series(arr_data["TEMP_ADJUSTED_ERROR"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED_QC': pd.Series(arr_data["PSAL_ADJUSTED_QC"].values.flatten(), index=ind),  
                  'PSAL_ADJUSTED_ERROR':pd.Series(arr_data["PSAL_ADJUSTED_ERROR"].values.flatten(), index=ind),  
                  'JULD': pd.Series(juld.flatten(), index=ind),  
                  'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                  'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                  'POSITION_QC': pd.Series(posqc.flatten(), index=ind)  }
            df_i = pd.DataFrame(df_i)
            
            #pdb.set_trace()
            df = df.append(df_i, ignore_index=True)
            del(arr_data)
    return df


In [13]:
dfm = create_merged_df(filenamesMerged[:])

In [14]:
len(dfm)

7948309

In [26]:
dfm.index

RangeIndex(start=0, stop=5586000, step=1)

In [15]:
dfm['TEMP_ADJUSTED_QC'] = dfm['TEMP_ADJUSTED_QC'].str.decode("utf-8")
dfm['PRES_ADJUSTED_QC'] = dfm['PRES_ADJUSTED_QC'].str.decode("utf-8")
dfm['PSAL_ADJUSTED_QC'] = dfm['PSAL_ADJUSTED_QC'].str.decode("utf-8")

In [16]:
dfm["JULD"] = pd.to_datetime(dfm['JULD'],infer_datetime_format=True)

In [17]:
dfm["DEPTH"] = gsw.z_from_p(dfm["PRES_ADJUSTED"], dfm["LATITUDE"])

In [18]:
SA = gsw.SA_from_SP(dfm['PSAL_ADJUSTED'],dfm['PRES_ADJUSTED'],dfm['LONGITUDE'],dfm['LATITUDE'])
CT = gsw.CT_from_t(SA, dfm['TEMP_ADJUSTED'], dfm['PRES_ADJUSTED'])
dfm['DENSITY_INSITU'] = gsw.density.rho(SA, CT, dfm['PRES_ADJUSTED'])
dfm['POT_DENSITY'] = gsw.density.sigma0(SA, CT)
dfm['CTEMP'] = CT
dfm['SA'] = SA
del(SA, CT)

In [19]:
mask_notbad_temp = ~(dfm['TEMP_ADJUSTED_QC'].str.contains('4'))
mask_notbad_sal = ~(dfm['PSAL_ADJUSTED_QC'].str.contains('4'))
mask_notbad_pres = ~(dfm['PRES_ADJUSTED_QC'].str.contains('4'))
mask_below60S = (dfm['LATITUDE'] < -60)
mask_all = (mask_notbad_pres & mask_notbad_sal & mask_notbad_temp & mask_below60S)

In [20]:
dfmg = dfm[mask_all]

In [41]:
dfmg['DIST_GLINE'] = np.empty(len(dfmg))

grndLine = gpd.read_file("/media/data/Datasets/Shapefiles/AntarcticGroundingLine/GSHHS_f_L6.shp")

/home/km/anaconda2/envs/py35new/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## The following distance computation is a very compute intensive and memory intensive process, ensure adequate RAM (> 16GB)

### Else, use the dfmg.csv file that has already been generated using the scripts in this notebook

In [42]:
def find_distance_to_groundingline(gdf, grndLine):
    indices = gdf.index
    df_lonlat = np.array([[gdf.loc[indices[0], 'LONGITUDE'],  gdf.loc[indices[0], 'LATITUDE'] ]])
    point = Point(df_lonlat[0])
    closest_polygon_ind = np.argmin(grndLine.geometry[:].distance(point))

    poly_points = np.array(list(grndLine.geometry[closest_polygon_ind].exterior.coords))
    closest_gl_point_ind = np.argmin(cdist(poly_points , df_lonlat))
    
    distance = float(haversine(poly_points[closest_gl_point_ind, ::-1], df_lonlat[0, ::-1]))
    #print(distance, type(distance))
    gdf.loc[:, 'DIST_GLINE'] = distance
    return gdf

In [43]:
dfmg = dfmg.groupby(['LATITUDE', 'LONGITUDE']).apply(find_distance_to_groundingline, grndLine)

### Run the following only if you want a column with ECHODEPTH created
Every lon lat point in the dataframe will be assigned an echodepth based on the closest available point in the GEBCO 2014 bathymetry 

In [2]:
dfmg = pd.read_csv("dfmg.csv")
del dfmg['Unnamed: 0']
dfmg.loc[:,'JULD'] = pd.to_datetime(dfmg.loc[:, 'JULD'])

In [9]:
bathy = netcdf.netcdf_file('/media/data/Datasets/Bathymetry/GEBCO_2014_2D.nc', 'r') ## give path to bathymetry file
lons = bathy.variables['lon'][:]
lats = bathy.variables['lat'][:]
dfmg['ECHODEPTH'] = np.nan

def assign_echodepth(df):
    ind = df.index
    lon = df.loc[ind[0], 'LONGITUDE']
    lat = df.loc[ind[0],'LATITUDE']
    ind_lon = np.argmin(np.abs(lons - lon))
    ind_lat = np.argmin(np.abs(lats - lat))
    df.loc[:,'ECHODEPTH'] = bathy.variables['elevation'][ind_lat, ind_lon]

    return df
    
dfmg = dfmg.groupby(['LATITUDE', 'LONGITUDE']).apply(assign_echodepth)

In [44]:
## Plays a beep sound. Useful to notify once any script is done running
def play_beep():
    import os
    duration = 1  # second
    freq = 440  # Hz
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
play_beep()

In [15]:
len(dfmg)

3353511

In [ ]:
dfmg.loc[:, 
         ["PROFILE_NUMBER", "PSAL_ADJUSTED", "TEMP_ADJUSTED", "PRES_ADJUSTED", "LONGITUDE", "LATITUDE"]]\
          .to_csv("STPlonlat.csv")

# Neutral Densities ($\gamma_N$) is produced using the matlab script: "compute_gamman.m"

The matlab script produces gamman.csv which should then be integrated into the dfmg.csv

In [17]:
dfmg.to_csv("dfmg.csv")

In [46]:
len(dfmg)

3353511

In [48]:
len(dfmg.PROFILE_NUMBER.unique())

199169